In [3]:
import pandas as pd
import os
from datetime import datetime
from PIL import Image
from datetime import datetime, timedelta
import time as time_lib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
TIFF_FOLDER = 'C:\\Users\\Tee\\senior_project\\src\\SolarMap\\DataAndResult\\cloud_images\\cloudmask'
PLANT_FOLDER = 'C:\\Users\\Tee\\senior_project\\src\\SolarMap\\DataAndResult\\metadata'
TRASH_FOLDER = 'C:\\Users\\Tee\\senior_project\\src\\SolarMap\\DataAndResult\\cloud_images\\trash'

In [5]:
plant_df = pd.read_csv(os.path.join(PLANT_FOLDER,'plant_metadata.csv'))

In [7]:
cloud_list = os.listdir(TIFF_FOLDER)
len(cloud_list)

In [16]:
## clean cloud images ##
trash1 = []
trash2 = []
for file in cloud_list:
    I = Image.open(os.path.join(TIFF_FOLDER,file))
    imarray = np.array(I)
    imarray = imarray[:,:,:3]
    sum_row = np.max(imarray,axis=1)
    sum_col = np.max(imarray,axis=0)
    sum_row = np.max(sum_row,axis=1)
    sum_col = np.max(sum_col,axis=1)
    
    if len(sum_row[sum_row==0])>1:
        trash1.append(file)
    if len(sum_col[sum_col==0])>1:
        trash2.append(file)

In [17]:
# check number of defect folder
q = set(trash2)
q.update(set(trash1))
len(q)

1

In [18]:
# change folder of defect images
for file in trash1:
    os.rename(os.path.join(TIFF_FOLDER,file), os.path.join(TRASH_FOLDER,file))
for file in trash2:
    if file not in trash1:
        os.rename(os.path.join(TIFF_FOLDER,file), os.path.join(TRASH_FOLDER,file))

In [22]:
cloud_list = os.listdir(TIFF_FOLDER)
len(cloud_list)

2478

In [23]:
## map cloud and site position lat/long 5x5
data = []
k=0; ch = 0
for file_name in cloud_list:
    I = Image.open(os.path.join(TIFF_FOLDER, file_name))
    imarray = np.array(I)
    time = datetime.strptime(file_name.split('_')[-1].split(".")[0], '%Y%m%d%H%M') + timedelta(hours=7)
    k+=1
    print(k, sep=' ', end='\r', flush=True)
    for i in range(56):
        name = plant_df.iloc[i]['site']
        row = plant_df.iloc[i]['row']
        col = plant_df.iloc[i]['col']
        data.append({'site_name': name, 'Datetime': time, \
                     'CI': float(imarray[int(row), int(col), ch])/255,\
                    'CI_1': float(imarray[int(row)-2, int(col)-2, ch])/255,\
                    'CI_2': float(imarray[int(row)-2, int(col)-1, ch])/255,\
                    'CI_3': float(imarray[int(row)-2, int(col), ch])/255,\
                    'CI_4': float(imarray[int(row)-2, int(col)+1, ch])/255,\
                    'CI_5': float(imarray[int(row)-2, int(col)+2, ch])/255,\
                    'CI_6': float(imarray[int(row)-1, int(col)-2, ch])/255,\
                    'CI_7': float(imarray[int(row)-1, int(col)-1, ch])/255,\
                    'CI_8': float(imarray[int(row)-1, int(col), ch])/255,\
                    'CI_9': float(imarray[int(row)-1, int(col)+1, ch])/255,\
                    'CI_10': float(imarray[int(row)-1, int(col)+2, ch])/255,\
                    'CI_11': float(imarray[int(row), int(col)-2, ch])/255,\
                    'CI_12': float(imarray[int(row), int(col)-1, ch])/255,\
                    'CI_14': float(imarray[int(row), int(col)+1, ch])/255,\
                    'CI_15': float(imarray[int(row), int(col)+2, ch])/255,\
                    'CI_16': float(imarray[int(row)+1, int(col)-2, ch])/255,\
                    'CI_17': float(imarray[int(row)+1, int(col)-1, ch])/255,\
                    'CI_18': float(imarray[int(row)+1, int(col), ch])/255,\
                    'CI_19': float(imarray[int(row)+1, int(col)+1, ch])/255,\
                    'CI_20': float(imarray[int(row)+1, int(col)+2, ch])/255,\
                    'CI_21': float(imarray[int(row)+2, int(col)-2, ch])/255,\
                    'CI_22': float(imarray[int(row)+2, int(col)-1, ch])/255,\
                    'CI_23': float(imarray[int(row)+2, int(col), ch])/255,\
                    'CI_24': float(imarray[int(row)+2, int(col)+1, ch])/255,\
                    'CI_25': float(imarray[int(row)+2, int(col)+2, ch])/255})

In [24]:
cidf = pd.DataFrame(data)

In [25]:
cidf.to_csv('ci_cloudmask_update.csv')

In [38]:
col_name = ['site_name','Datetime']
for d in range(256):
    col_name.append(d)

In [49]:
## map cloud and site position lat/long 16x16
data = []
k=0
    
for file_name in cloud_list:
    I = Image.open(os.path.join(TIFF_FOLDER, file_name))
    imarray = np.array(I)
    time = datetime.strptime(file_name.split('_')[-1].split(".")[0], '%Y%m%d%H%M') + timedelta(hours=7)
    k+=1
    print(k, sep=' ', end='\r', flush=True)
    for i in range(56):
        name = plant_df.iloc[i]['site']
        row = plant_df.iloc[i]['row']
        col = plant_df.iloc[i]['col']
        cropped_img1 = imarray[row-8:row+8,col-8:col+8,0]
        pix1 = cropped_img1.reshape(1,256)[0].tolist()
        d = [name,time] + pix1
        data.append(d)
img_df = pd.DataFrame(data,columns=col_name)

In [69]:
img_df = pd.DataFrame(data,columns=col_name)

In [70]:
img_df

,site,datetime,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,ISL001,2022-03-23 09:20:00,137,142,141,144,136,135,136,145,...,104,112,126,128,134,107,108,104,113,120
1,ISL002,2022-03-23 09:20:00,105,105,107,106,100,108,104,99,...,108,125,119,116,110,120,132,136,136,137
2,ISL003,2022-03-23 09:20:00,96,95,96,93,97,104,106,101,...,149,151,136,131,142,147,142,153,157,179
3,ISL004,2022-03-23 09:20:00,119,116,128,113,110,109,128,129,...,138,144,153,165,167,171,180,203,213,211
4,ISL005,2022-03-23 09:20:00,198,163,166,159,144,137,130,121,...,154,144,147,162,157,158,161,152,148,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252163,ISL052,2022-09-24 08:50:00,112,117,122,125,130,131,132,133,...,147,149,147,145,148,146,145,148,147,148
252164,ISL053,2022-09-24 08:50:00,117,121,130,132,133,133,132,135,...,151,153,154,150,147,147,143,142,144,144
252165,ISL054,2022-09-24 08:50:00,117,121,130,132,133,133,132,135,...,151,153,154,150,147,147,143,142,144,144
252166,ISL055,2022-09-24 08:50:00,112,117,122,125,130,131,132,133,...,147,149,147,145,148,146,145,148,147,148


In [71]:
img_df.set_index('Datetime', inplace=True)
img_df.index = img_df.index.tz_localize('Asia/Bangkok')
img_df.iloc[:,1:] = img_df.iloc[:,1:]/255

In [74]:
img_df

,site_name,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-03-23 09:30:00+07:00,ISL001,0.537255,0.556863,0.552941,0.564706,0.533333,0.529412,0.533333,0.568627,0.560784,...,0.407843,0.439216,0.494118,0.501961,0.525490,0.419608,0.423529,0.407843,0.443137,0.470588
2022-03-23 09:30:00+07:00,ISL002,0.411765,0.411765,0.419608,0.415686,0.392157,0.423529,0.407843,0.388235,0.392157,...,0.423529,0.490196,0.466667,0.454902,0.431373,0.470588,0.517647,0.533333,0.533333,0.537255
2022-03-23 09:30:00+07:00,ISL003,0.376471,0.372549,0.376471,0.364706,0.380392,0.407843,0.415686,0.396078,0.454902,...,0.584314,0.592157,0.533333,0.513725,0.556863,0.576471,0.556863,0.600000,0.615686,0.701961
2022-03-23 09:30:00+07:00,ISL004,0.466667,0.454902,0.501961,0.443137,0.431373,0.427451,0.501961,0.505882,0.545098,...,0.541176,0.564706,0.600000,0.647059,0.654902,0.670588,0.705882,0.796078,0.835294,0.827451
2022-03-23 09:30:00+07:00,ISL005,0.776471,0.639216,0.650980,0.623529,0.564706,0.537255,0.509804,0.474510,0.509804,...,0.603922,0.564706,0.576471,0.635294,0.615686,0.619608,0.631373,0.596078,0.580392,0.568627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-24 09:00:00+07:00,ISL052,0.439216,0.458824,0.478431,0.490196,0.509804,0.513725,0.517647,0.521569,0.533333,...,0.576471,0.584314,0.576471,0.568627,0.580392,0.572549,0.568627,0.580392,0.576471,0.580392
2022-09-24 09:00:00+07:00,ISL053,0.458824,0.474510,0.509804,0.517647,0.521569,0.521569,0.517647,0.529412,0.537255,...,0.592157,0.600000,0.603922,0.588235,0.576471,0.576471,0.560784,0.556863,0.564706,0.564706
2022-09-24 09:00:00+07:00,ISL054,0.458824,0.474510,0.509804,0.517647,0.521569,0.521569,0.517647,0.529412,0.537255,...,0.592157,0.600000,0.603922,0.588235,0.576471,0.576471,0.560784,0.556863,0.564706,0.564706


In [75]:
img_df.to_csv('ci_cloudmask.csv') # export to csv